<a href="https://colab.research.google.com/github/sgcortes/UrbanAccessibility/blob/main/CoordsAbsolutasPasosCebraB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Conversión a Coords Absolutas de las coordenadas de los pasos de cebra detectados


In [1]:
!pip install pyshp
!pip install owslib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 kB 1.5 MB/s eta 0:00:00


### Lectura de fichero json proveniente de la detección.

In [2]:
import json
#from pyproj import Transformer
import numpy as np

# Leer el archivo JSON
with open('/content/tesela_mapa_row4_col1.json', 'r') as f:
    data = json.load(f)

# Extraer las coordenadas de los pasos de cebra detectados
coords = []
for obj in data['predictions']:
    if obj['class'] == 'zebra-cross':
        coords.append((obj['x'], obj['y']))

# Imprimir las coordenadas de los pasos de cebra detectados
print('Coordenadas de los pasos de cebra detectados:')
for coord in coords:
    print(coord)
print(len(coords))

unos = np.ones((1,len(coords)))

coords = np.vstack(((np.array(coords).T, unos)))
print(coords)

FileNotFoundError: [Errno 2] No such file or directory: '/content/tesela_mapa_row4_col1.json'

### Lectura de los ficheros mundo de cada tesela de imagen y transforamción de coords

In [ ]:
# Definir la transformación afín
import numpy as np

# Leer los valores del archivo mundo
with open('/content/tesela_mapa_row4_col1.jgw', 'r') as f:
    mundo = [float(x.strip()) for x in f.readlines()]

# Definir la matriz de transformación afín
affine_transform = np.array([[mundo[0], 0, mundo[4]], [0, mundo[3], mundo[5]], [0, 0, 1]])

# Imprimir la matriz de transformación afín
print('Matriz de transformación afín:')
print(affine_transform)


In [ ]:
# Aplicar la transformación afín a las coordenadas locales
abs_coords = affine_transform @ coords
abs_coords = abs_coords[0:2,:].T
print(abs_coords)


### Creación de fichero de puntos con centro de paso de cebra

In [ ]:
### Dado que no se representa bien el shapefile sobre el mapa WMS en EPSG 25830 voy a crear un fichero de texto csv con las coords
import csv
import numpy as np

# Suponiendo que 'abs_coords' es una matriz de NumPy
# Por ejemplo: abs_coords = np.array([[x1, y1], [x2, y2], ..., [xn, yn]])

# Convertir la matriz de NumPy a una lista de tuplas
coords_list = abs_coords.tolist()

# Crear un nuevo archivo csv
with open('coords.csv', 'w', newline='') as f:
    writer = csv.writer(f)

    # Escribir los encabezados de las columnas
    writer.writerow(['X', 'Y'])

    # Escribir los valores de la matriz
    for coord in coords_list:
        writer.writerow(coord)

print("Archivo csv creado con éxito.")


In [ ]:
import shapefile

# Suponiendo que 'coordinates' es una lista de tuplas (x, y)
# Por ejemplo: coordinates = [(x1, y1), (x2, y2), ..., (xn, yn)]
coordinates = abs_coords  #

# Crear un nuevo shapefile en memoria
w = shapefile.Writer('PasosCebra.shp', shapeType=shapefile.POINT)

# Para cada punto, crear un punto en el shapefile
for coord in coordinates:
    w.point(*coord)

# Agregar campos al shapefile. Aquí se agregan dos campos de ejemplo.
w.field('X', 'F', decimal=10)
w.field('Y', 'F', decimal=10)

# Agregar registros para cada punto con sus coordenadas x, y
for coord in coordinates:
    w.record(*coord)
# Definir el sistema de coordenadas del shapefile como EPSG 25830
w.prj = open('PasosCebra.prj', 'w')
w.prj.write('PROJCS["ETRS89 / UTM zone 30N",GEOGCS["ETRS89",DATUM["European_Terrestrial_Reference_System_1989",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6258"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4258"]],UNIT["metre",1,AUTHORITY["EPSG","9001"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-3],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],AUTHORITY["EPSG","25830"],AXIS["Easting",EAST],AXIS["Northing",NORTH]]')
w.prj.close()

# Guardar el shapefile
w.close()
# Guardar el shapefile
w.close()

print("Archivo shapefile creado con éxito.")


### Visualizando los pasos de cebre sobra el mapa

In [ ]:
import owslib.wms
import io
from PIL import Image
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

## Estimando los limites para la orto

df = pd.read_csv('coords.csv', header=0, names=['X', 'Y'])
#print(df)
min_X = df['X'].min()
max_X = df['X'].max()

min_Y = df['Y'].min()
max_Y = df['Y'].max()

margen = 150

bbox=(min_X-margen, min_Y-margen, max_X+margen, max_Y+margen)
print(bbox)
size0 = max_X+margen - (min_X-margen)
size1 = max_Y+margen - (min_Y-margen)

# Conexión al servidor WMS del CNIG
wms = owslib.wms.WebMapService('https://www.ign.es/wms-inspire/pnoa-ma', version='1.3.0')

# Obtener la ortofoto deseada
img = wms.getmap(layers=['OI.OrthoimageCoverage'], srs='EPSG:25830', bbox=bbox, size=(size0, size1), format='image/png', transparent=True)
# bbox=(min_X-margen, min_Y-margen, max_X+margen, max_Y+margen)

img_bytes = io.BytesIO(img.read())
img = Image.open(img_bytes)


# Crear un GeoDataFrame a partir del DataFrame
# Cargar los datos del archivo csv

gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['X'], df['Y']), crs='EPSG:25830')
print(gdf)

# Crear una figura y un eje utilizando geopandas
fig, ax = plt.subplots(figsize=(10, 10))

# Agregar la ortofoto al eje
ax.imshow(img, extent=bbox, aspect='equal')
#extent = [min_X-margen, min_Y-margen, max_X+margen, max_Y+margen]

# Agregar los puntos al eje
gdf.plot(ax=ax, markersize=150, color='red')
plt.grid('on')
# Agregar etiquetas a los ejes
ax.set_xlabel('Coordenada X')
ax.set_ylabel('Coordenada Y')

# Agregar un título a la figura
ax.set_title('Ortofoto del PNOA con puntos en EPSG 25830')

plt.show()


In [ ]:
!pip install cartopy

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import owslib.wms
import io
from PIL import Image

# Cargar los datos
df = pd.read_csv('coords.csv', header=0, names=['X', 'Y'])
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['X'], df['Y']), crs='EPSG:25830')

# Calcular los límites con un margen
margen = 150
min_X = df['X'].min() - margen
max_X = df['X'].max() + margen
min_Y = df['Y'].min() - margen
max_Y = df['Y'].max() + margen
bbox = (min_X, min_Y, max_X, max_Y)
print(bbox)
# Conexión al servidor WMS del CNIG y obtención de la ortofoto
wms = owslib.wms.WebMapService('https://www.ign.es/wms-inspire/pnoa-ma', version='1.3.0')

# Tamaño de la imagen (puede necesitar ajuste para mantener la relación de aspecto)
size = (int(2 * (max_X - min_X)), int(2 * (max_Y - min_Y)))

# Obtener la ortofoto deseada
img_request = wms.getmap(layers=['OI.OrthoimageCoverage'], srs='EPSG:25830', bbox=bbox, size=size, format='image/png', transparent=True)

# Escribir la imagen a un archivo
with open('ortofoto.png', 'wb') as f:
    f.write(img_request.read())



# Crear una figura y un eje con GeoAxes en la proyección UTM
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={'projection': ccrs.UTM(zone=30)})

# Establecer la extensión de los ejes
ax.set_extent([min_X, max_X, min_Y, max_Y], crs=ccrs.UTM(zone=30))

# Graficar la ortofoto como fondo
ax.imshow(img, origin='upper', extent=bbox, transform=ccrs.UTM(zone=30), interpolation='nearest')

# Graficar los puntos sobre la ortofoto
#gdf.plot(ax=ax, markersize=150, color='red', transform=ccrs.UTM(zone=30))

# Ajustar etiquetas y títulos
ax.set_xlabel('Coordenada X')
ax.set_ylabel('Coordenada Y')
ax.set_title('Ortofoto del PNOA con puntos en EPSG 25830')

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
from PIL import Image
import cartopy.crs as ccrs

# Asumiendo que ya tienes tu GeoDataFrame 'gdf' listo y en el CRS correcto
# gdf = ...

# Cargar la imagen de la ortofoto
img = Image.open('ortofoto.png')

# Crear una figura y un eje con GeoAxes en la proyección UTM
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={'projection': ccrs.UTM(zone=30)})

# Establecer la extensión de los ejes
# Asegúrate de que bbox sea la misma que usaste para obtener la ortofoto
ax.set_extent([min_X, max_X, min_Y, max_Y], crs=ccrs.UTM(zone=30))

# Graficar la ortofoto como fondo
# La extent debe coincidir con las coordenadas reales de tu ortofoto
ax.imshow(img, origin='upper', extent=bbox, transform=ccrs.UTM(zone=30), interpolation='nearest')

# Graficar los puntos sobre la ortofoto
# Asegúrate de que 'gdf' esté en el mismo CRS que los ejes de cartopy
gdf.plot(ax=ax, markersize=5, color='red', transform=ccrs.UTM(zone=30))

# Ajustar etiquetas y títulos si es necesario
ax.set_xlabel('Coordenada X')
ax.set_ylabel('Coordenada Y')
ax.set_title('Ortofoto del PNOA con puntos en EPSG 25830')

plt.show()
